# tensorflow 实现简单线性回归

In [3]:
# tensorflow environment
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

# refer to d2l
from d2l import tensorflow as d2l

# using type hint from third party tools
# deprecated since python3.9
# from typing import list

# ensure GPU available
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-09-06 20:51:14.092704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-06 20:51:14.193125: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-06 20:51:14.194728: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


## 生成数据

In [33]:
def synthetic_data(w:list[float],b:float,num:int):
    """
    creat data
    """
    x = tf.zeros((num,w.shape[0]))
    x += tf.random.normal(shape=x.shape) # noise
 
    y= tf.matmul(x,tf.reshape(w,(-1,1))) + b
    y += tf.random.normal(shape=y.shape,stddev=0.01)
    y = tf.reshape(y,(-1,1))
    return x,y


In [35]:
# weights
w = tf.constant([1,2.1,3],dtype=float)
# bias
b = -1
n = 1000

x,y=synthetic_data(w,b,n)
print("features:",x[0])
print("labels:",y[0])

features: tf.Tensor([-0.7729939  -0.02612727 -1.132695  ], shape=(3,), dtype=float32)
labels: tf.Tensor([-5.224094], shape=(1,), dtype=float32)


## 读取数据

In [37]:
def data_iter(batch_size,features,lables):
    """通过yield生成迭代器"""
    nums=len(features)
    # 随机读取样本
    indices = list(range(nums))
    random.shuffle(indices)
    for i in range(0,nums,batch_size):
        j = tf.constant(indices[i:min(i+batch_size,nums)])
        yield tf.gather(features,j),tf.gather(lables,i)
    pass

In [39]:
batch_size = 10

for x,y in data_iter(batch_size,x,y):
    print(x,"\n",y)
    break

tf.Tensor(
[[ 0.97910404  0.7734633  -0.4179601 ]
 [-0.04344636 -1.026765    0.16779661]
 [ 0.20594802 -1.3332775   1.5568743 ]
 [-0.5375585   0.02788551 -1.2643732 ]
 [-1.817231   -1.5623933   0.59103996]
 [ 0.20396647  0.7046695   0.43296498]
 [ 0.9863119  -2.0040646   0.91954654]
 [ 1.6995543  -0.21245213  0.8217019 ]
 [-0.5274577  -1.1235367   1.7480526 ]
 [ 1.6953299   0.57285887  0.23089398]], shape=(10, 3), dtype=float32) 
 tf.Tensor(-1.7972207, shape=(), dtype=float32)


## 建立线性回归模型

In [ ]:
# 初始化模型参数
tw = tf.Variable(tf.random.normal(shape=(3,1),mean=0,stddev=0.01),trainable=True)
tb = tf.Variable(tf.zeros(1))

## 选择合适的损失函数和优化器